<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20Region%20segmentation%20in%20synthetic%20lunar%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## 1. Installation Instructions



## 2. Use trained model to segment traversable regions in synthetic lunar imagery



## 3. How to train a custom segmenter using "artificial-lunar-rocky-landscape-dataset"

# About the networks


1. UNet
  - https://arxiv.org/abs/1505.04597
  - https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47
  - https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5

2. FPN
  - http://openaccess.thecvf.com/content_cvpr_2017/papers/Lin_Feature_Pyramid_Networks_CVPR_2017_paper.pdf
  - https://towardsdatascience.com/review-fpn-feature-pyramid-network-object-detection-262fc7482610
  - https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c


3. PSPNet
  - https://arxiv.org/abs/1612.01105
  - https://towardsdatascience.com/review-pspnet-winner-in-ilsvrc-2016-semantic-segmentation-scene-parsing-e089e5df177d
  - https://developers.arcgis.com/python/guide/how-pspnet-works/


4. Linknet
  - https://arxiv.org/pdf/1707.03718.pdf
  - https://neptune.ai/blog/image-segmentation-tips-and-tricks-from-kaggle-competitions

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/9_segmentation_models/installation
     
 - Select the right requirements file and run
 
     - cat requirements_cuda9.0.txt | xargs -n 1 -L 1 pip install

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
! cd Monk_Object_Detection/9_segmentation_models/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install


# For Local systems and cloud select the right CUDA version
#! cd Monk_Object_Detection/9_segmentation_models/installation && cat requirements_cuda10.0.txt | xargs -n 1 -L 1 pip install

# Use already trained model for demo

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'traversable_land': 0, 
    'space': 1, 
    'rocks1': 2, 
    'rocks2': 3
};
classes_to_train = ['traversable_land', 'space', 'rocks1', 'rocks2'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[720, 480])

In [ ]:
# Download trained model

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lR6pTN9hL4ZU1slfutux1Nah5s_FPhk8' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1lR6pTN9hL4ZU1slfutux1Nah5s_FPhk8" -O seg_lunar_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq seg_lunar_trained.zip

In [ ]:
gtf.Model_Params(model="Unet", backbone="efficientnetb3", path_to_model='seg_lunar_trained/best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
gtf.Predict("seg_lunar_trained/test/7.png", vis=True);

# Training your own segmenter 

## Dataset Credits
    - https://www.kaggle.com/romainpessia/artificial-lunar-rocky-landscape-dataset

In [ ]:
! pip install kaggle

In [ ]:
! kaggle datasets download romainpessia/artificial-lunar-rocky-landscape-dataset

In [ ]:
! unzip -qq artificial-lunar-rocky-landscape-dataset.zip -d lunar_dataset

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread("lunar_dataset/real_moon_images/g_PCAM12.png", 1)
img.shape

In [ ]:
np.unique(img)

In [ ]:
from IPython.display import Image
Image(filename="lunar_dataset/real_moon_images/PCAM12.png") 

In [ ]:
np.unique(img[:, :, 2])

In [ ]:
img[img[:, :, 2] > 0] == 255
plt.imshow(img[:, :, 2])
plt.title('my picture')
plt.show()

In [ ]:
img[img[:, :, 1] > 0] == 255
plt.imshow(img[:, :, 1])
plt.title('my picture')
plt.show()

In [ ]:
img[img[:, :, 0] > 0] == 255
plt.imshow(img[:, :, 0])
plt.title('my picture')
plt.show()

In [ ]:
img_rgb = cv2.imread("lunar_dataset/images/ground/ground0001.png", 1);
img_mask = cv2.imread("lunar_dataset/images/ground/ground0001.png", 0);

img_mask[img_rgb[:, :, 2] > 0] = 0;
img_mask[img_rgb[:, :, 1] > 0] = 1;
img_mask[img_rgb[:, :, 0] > 0] = 2;

cv2.imwrite("output.jpg", img_mask);
cv2.imwrite("output_vis.jpg", img_mask*50);

In [ ]:
from IPython.display import Image
Image(filename="output_vis.jpg") 

In [ ]:
import os

grd_list = sorted(os.listdir("lunar_dataset/images/ground/"))
img_list = sorted(os.listdir("lunar_dataset/images/render/"))

In [ ]:
os.mkdir("lunar_dataset/modified/");
os.mkdir("lunar_dataset/modified/train");
os.mkdir("lunar_dataset/modified/labels");
os.mkdir("lunar_dataset/modified/labels_visible")

In [ ]:
from tqdm.notebook import tqdm

for i in tqdm(range(len(grd_list))):
    img_path = "lunar_dataset/images/render/" + img_list[i];
    grd_path = "lunar_dataset/images/ground/" + grd_list[i];
    
    os.system("cp " + img_path + " lunar_dataset/modified/train/" + str(i) + ".png");
    
    img_rgb = cv2.imread(grd_path, 1);
    img_mask = cv2.imread(grd_path, 0);

    img_mask[img_rgb[:, :, 2] == 0] = 0;
    img_mask[img_rgb[:, :, 2] > 0] = 1;
    img_mask[img_rgb[:, :, 1] > 0] = 2;
    img_mask[img_rgb[:, :, 0] > 0] = 3;

    cv2.imwrite("lunar_dataset/modified/labels/" + str(i) + ".png", img_mask);
    cv2.imwrite("lunar_dataset/modified/labels_visible/" + str(i) + ".png", img_mask*50);

In [ ]:
img_rgb.shape

In [ ]:
from IPython.display import Image
img_rgb = cv2.imread("lunar_dataset/modified/labels/100.png", 0);
np.unique(img_rgb)
Image(filename="lunar_dataset/modified/labels_visible/100.png")

In [ ]:
from IPython.display import Image
Image(filename="lunar_dataset/modified/train/100.png") 

# Monk Format

## Dataset Directory Structure

      root_dir
          |
          | 
          |         
          |----train_img_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on) 
          |
          |----train_mask_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_img_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_mask_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)

          
          

# Sample Dataset Credits

    credits: https://github.com/alexgkendall/SegNet-Tutorial

In [ ]:
DATA_DIR = 'lunar_dataset/modified/'

# Training

In [ ]:
import os
import sys
sys.path.append("../../9_segmentation_models/lib/");

In [ ]:
from train_segmentation import Segmenter

In [ ]:
gtf = Segmenter();

In [ ]:
img_dir = "lunar_dataset/modified/train/";
mask_dir = "lunar_dataset/modified/labels/";

In [ ]:
classes_dict = {
    'traversable_land': 0, 
    'space': 1, 
    'big_rocks': 2, 
    'small_rocks': 3
};
classes_to_train = ['traversable_land', 'space', 'big_rocks', 'small_rocks'];

In [ ]:
gtf.Train_Dataset(img_dir, mask_dir, classes_dict, classes_to_train)

In [ ]:
img_dir = "lunar_dataset/modified/train/";
mask_dir = "lunar_dataset/modified/labels/";

In [ ]:
gtf.Val_Dataset(img_dir, mask_dir)

In [ ]:
gtf.List_Backbones();

In [ ]:
gtf.Data_Params(batch_size=2, backbone="efficientnetb3", image_shape=[720, 480])

In [ ]:
gtf.List_Models();

In [ ]:
gtf.Model_Params(model="Unet")

In [ ]:
gtf.Train_Params(lr=0.0001)

In [ ]:
gtf.Setup();

In [ ]:
gtf.Train(num_epochs=2);

In [ ]:
gtf.Visualize_Training_History();

# Inference

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'traversable_land': 0, 
    'space': 1, 
    'big_rocks': 2, 
    'small_rocks': 3
};
classes_to_train = ['traversable_land', 'space', 'big_rocks', 'small_rocks'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[720, 480])

In [ ]:
gtf.Model_Params(model="Unet", backbone="efficientnetb3", path_to_model='best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
import cv2
img = cv2.imread("lunar_dataset/modified/train/10.png", 1)
img = cv2.resize(img, (480, 480))
cv2.imwrite("output.jpg", img)
img.shape

In [ ]:
gtf.Predict("lunar_dataset/modified/train/10.png", vis=True);